### After the seller buy the device from manufacture, seller needs proof of having the device

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_credential


### Go to Seller Notebook to build connection

In [4]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'connection_id': 'e812df45-5cab-4e9b-a099-cd801de4fd33', 'accept': 'manual', 'invitation_key': 'EzJK5B9Whm1rm4utkVJj8HpevUg6ppU3ete5YfQLRKTn', 'invitation_mode': 'once', 'routing_state': 'none', 'their_did': 'WGBGMeCT19cC5Thdjey9fa', 'created_at': '2023-01-03 00:10:27.158633Z', 'state': 'active', 'their_role': 'inviter', 'updated_at': '2023-01-03 00:10:47.369114Z', 'my_did': '5mjER6QMs8dEZdfJvfNcXZ', 'their_label': 'Bob', 'rfc23_state': 'completed', 'request_id': '84a406b6-85c3-49c1-afca-681a76aa2a3d'}]
Connection : {'connection_id': 'e812df45-5cab-4e9b-a099-cd801de4fd33', 'accept': 'manual', 'invitation_key': 'EzJK5B9Whm1rm4utkVJj8HpevUg6ppU3ete5YfQLRKTn', 'invitation_mode': 'once', 'routing_state': 'none', 'their_did': 'WGBGMeCT19cC5Thdjey9fa', 'created_at': '2023-01-03 00:10:27.158633Z', 'state': 'active', 'their_role': 'inviter', 'updated_at': '2023-01-03 00:10:47.369114Z', 'my_did': '5mjER6QMs8dEZdfJvfNcXZ', 'their_label': 'Bob', 'rfc23_state': 'completed', 'request_i

In [9]:
# Define you schema name - must be unique on the ledger
schema_name = "Device_schema3"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["ProductionDate", "ExpirationDate", "Manufacturer", "Owner", "Type", "Status"]

In [10]:
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

PQRXDxdGqQGSZ8z69p4xZP:2:Device_schema3:0.0.1


In [11]:
response = await agent_controller.definitions.write_cred_def(schema_id, support_revocation=True)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:40:default


In [12]:
import time

ProductionTime = int(time.time())
ExpirationTime = ProductionTime + 31536000

credential_attributes = [
    {"name": "ProductionDate", "value": str(ProductionTime)},
    {"name": "ExpirationDate", "value": str(ExpirationTime)},
    {"name": "Manufacturer", "value": "GOOD_COMPANY"},
    {"name": "Owner", "value": "GOOD_SELLER"},
    {"name": "Type", "value": "Dentistry"},
    {"name": "Status", "value": "1"}
]
print(credential_attributes)

[{'name': 'ProductionDate', 'value': '1672706006'}, {'name': 'ExpirationDate', 'value': '1704242006'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]


In [13]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 5697823d-3a42-4322-b5e4-de320ab94d41, role: issuer, state: offer_sent
Offering: [{'name': 'ProductionDate', 'value': '1672706006'}, {'name': 'ExpirationDate', 'value': '1704242006'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]
Credential exchange 5697823d-3a42-4322-b5e4-de320ab94d41, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 5697823d-3a42-4322-b5e4-de320ab94d41, role: issuer, state: request_received
Offering: [{'name': 'ProductionDate', 'value': '1672706006'}, {'name': 'ExpirationDate', 'value': '1704242006'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]
Handle Credentials
Credential exchange 5697823d-3a42-4322-b5e4-de320ab94d41, role: issuer, state: credential_issued
Offering

### Go to seller notebook to receive credential

In [14]:
response = await agent_controller.terminate()
print(response)

None
